# Scotch Whisky Exploration: Data Wrangling

In [1]:
import numpy as np
import pandas as pd

import re
import spacy


In [2]:
whiskydf_raw = pd.read_json("whiskeyscraper/MoM_whiskydata.json")

In [3]:
whiskydf_raw.head(2)

,name,nose,palate,finish,description,region,style,distillery,bottler,age,alcohol,maturation,chill_filter,cask_strength
0,Singleton of Dufftown 12 Year Old,"Malty with cereal/barley sweetness, buttery to...",Orange zest spiciness perks up a malty core of...,"Oaky, rich with good length, some fruit lingers.","A straightforward, nutty and malty single malt...",Speyside Whisky,Single Malt Whisky,Dufftown,Dufftown,12 year old Whisky,40.0%,NaN,NaN,NaN
1,Laphroaig 10 Year Old Sherry Oak Finish,"Smoked meats, maple syrup, BBQ lemon, charred ...","More roasted cedar and peat smoke, with a hint...",A balanced finish of sherried sweetness and sm...,Smoke and sherry here from Laphroaig! The lege...,Islay Whisky,Single Malt Whisky,Laphroaig,Laphroaig,10 year old Whisky,48.0%,NaN,NaN,NaN


## Initial Data Clean: 
#### Dropping observations with no reviews or missing taste,nose, and finish notes.
#### Coverting/cleaning numeric columns. 

In [4]:
whiskydf_raw.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14143 entries, 0 to 14142
Data columns (total 14 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   name           14143 non-null  object
 1   nose           8553 non-null   object
 2   palate         8552 non-null   object
 3   finish         8488 non-null   object
 4   description    14143 non-null  object
 5   region         14143 non-null  object
 6   style          14143 non-null  object
 7   distillery     14143 non-null  object
 8   bottler        14142 non-null  object
 9   age            10667 non-null  object
 10  alcohol        14142 non-null  object
 11  maturation     2376 non-null   object
 12  chill_filter   479 non-null    object
 13  cask_strength  1022 non-null   object
dtypes: object(14)
memory usage: 1.5+ MB


### Let's keep the subset of whiskeys where there are tasting notes (e.g. where nose, palate, and finish are not null)

In [5]:
tastingnote_cols = ['nose', 'palate', 'finish']

whiskydf_raw[tastingnote_cols].isna().all() == True

nose      False
palate    False
finish    False
dtype: bool

Takes subset where Master of Malt bros have created tasting notes on the whiskey. 

In [6]:
whisky_df =  whiskydf_raw.dropna(how = "any", axis = 0, subset= tastingnote_cols)

In [7]:
whisky_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8485 entries, 0 to 14141
Data columns (total 14 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   name           8485 non-null   object
 1   nose           8485 non-null   object
 2   palate         8485 non-null   object
 3   finish         8485 non-null   object
 4   description    8485 non-null   object
 5   region         8485 non-null   object
 6   style          8485 non-null   object
 7   distillery     8485 non-null   object
 8   bottler        8485 non-null   object
 9   age            6557 non-null   object
 10  alcohol        8485 non-null   object
 11  maturation     1613 non-null   object
 12  chill_filter   403 non-null    object
 13  cask_strength  550 non-null    object
dtypes: object(14)
memory usage: 994.3+ KB


Our scraper extracted a good amount of data and it'll be good to see whether there are some entries that don't belong (i.e styles that are not single malt, etc.)

In [8]:
whisky_df['style'].unique()

array(['Single Malt Whisky'], dtype=object)

Alright, so these are all single malt whiskies. Good. But since there's only one value the column carries no information. We'll drop it.  Also there seem to be no disambiguations/mispellings in the Scotch whisky region names. Each unique entry corresponds to a different Scotch whisky making region.

In [9]:
whisky_df.region.unique()

array(['Speyside Whisky', 'Islay Whisky', 'Highland Whisky',
       'Island Whisky', 'Scotch Whisky', 'Lowland Whisky',
       'Campbeltown Whisky', 'Other Scotch Whisky'], dtype=object)

In [10]:
whisky_df = whisky_df.drop(columns = ['style'])
print(whisky_df.columns)

Index(['name', 'nose', 'palate', 'finish', 'description', 'region',
       'distillery', 'bottler', 'age', 'alcohol', 'maturation', 'chill_filter',
       'cask_strength'],
      dtype='object')


#### Age column
Now we'll tackle the age column. Generally, the whiskies are matured for years in casks before bottling. The age corresponds to the aging before bottling. But let's check whether there are other units (months, etc.) of aging buried in the data.

In [11]:
whisky_df.age.head(3)

0    12 year old Whisky
1    10 year old Whisky
2    15 year old Whisky
Name: age, dtype: object

The second word is the aging unit (year). Let's extract this second word and see if anything else pops up.

In [12]:
whisky_df.age.str.split().str[1].unique()

array(['year', nan], dtype=object)

Nope. Either the age is recorded in years or the entry corresponds to a whisky with no age statement. Let's process this understanding that age is in years and convert the column to numeric.

In [13]:
#convert whiskey age to numeric.
whisky_df.age = whisky_df.age.str.split(' ').str[0].astype('float')

In [14]:
np.sort(whisky_df.age.unique())

array([   3.,    4.,    5.,    6.,    7.,    8.,    9.,   10.,   11.,
         12.,   13.,   14.,   15.,   16.,   17.,   18.,   19.,   20.,
         21.,   22.,   23.,   24.,   25.,   26.,   27.,   28.,   29.,
         30.,   31.,   32.,   33.,   34.,   35.,   36.,   37.,   38.,
         39.,   40.,   41.,   42.,   43.,   44.,   45.,   46.,   47.,
         48.,   49.,   50.,   51.,   52.,   54.,   55.,   56.,   60.,
         62.,   64.,   65.,   71.,   78.,  105., 2003.,   nan])

105 and 2003 year old whisky are possible outliers. Let's investigate more closely.

In [15]:
whisky_df[whisky_df.age == 105]

,name,nose,palate,finish,description,region,distillery,bottler,age,alcohol,maturation,chill_filter,cask_strength
9219,Aisla T'Orten 105 Year Old 1906 - Liquid Histo...,The most unique bouquet I’ve ever experienced....,Heaven is spelt “T-O-R-T-E-N”. This conjures u...,To say this was long would be an understatemen...,The world's oldest whisky. Read more about thi...,Highland Whisky,Aisla T'Orten,Master of Malt,105.0,40.7%,Sherry,NaN,NaN


In [16]:
whisky_df[whisky_df.age == 105].description.values

array(["The world's oldest whisky. Read more about this extraordinary liquid history over on the Master of Malt blog . OK everyone, we have to come clean! The miraculous discovery of this 105 year old whisky may have had something to do with its launch on April Fools’ Day 2011. It’s also possible that some sneaky anagrams were used in some of the names in the story: Allie Sisell (the discoverer of the cask):  Aethenias Simonvent (the distillery’s founder):  And there’s a good chance that if you rearrange the letters in Aisla T’Orten distillery you get:"],
      dtype=object)

It's an April Fools joke! Gaaaarrrrr!!!! Remove this.

In [17]:
whisky_df = whisky_df[~(whisky_df.age == 105)]
print(whisky_df.age.unique())

[  12.   10.   15.   nan   18.   25.   14.   21.   13.   16.    8.    5.
   11.   26.   23.   43.   28.   24.    9.   30.   29.   42.   19.   27.
   20.   17.    7.   40.    6.   22.   45.    3.   37.   35.   31.   64.
   50.   46.   36.   32.   44.   56.   38.   34.   33.   62.    4.   41.
   39. 2003.   60.   54.   55.   78.   52.   49.   47.   48.   51.   71.
   65.]


In [18]:
whisky_df[whisky_df.age == 2003].description.values

array(["Batch 4 of Benromach's Origins range, bottled in 2013, is matured entirely in first fill Port pipes and peated to a level of 8ppm. The idea behind the range is to highlight how changing different factors during production results in different characteristics in the final spirit."],
      dtype=object)

This batch was started on 2003 but bottled in 2013, so we'll change the age value to 10 years. 

In [19]:
whisky_df.loc[whisky_df.age == 2003, "age"] = 10

In [20]:
# check if we have removed the outlier. 
whisky_df.age.unique()

array([12., 10., 15., nan, 18., 25., 14., 21., 13., 16.,  8.,  5., 11.,
       26., 23., 43., 28., 24.,  9., 30., 29., 42., 19., 27., 20., 17.,
        7., 40.,  6., 22., 45.,  3., 37., 35., 31., 64., 50., 46., 36.,
       32., 44., 56., 38., 34., 33., 62.,  4., 41., 39., 60., 54., 55.,
       78., 52., 49., 47., 48., 51., 71., 65.])

#### Alcohol column
Yes. Now we convert the alcohol column to numeric value, we'll also change the name from alcohol to ABV as this is known to be in percent.

In [21]:
whisky_df.alcohol = whisky_df.alcohol.str.replace('%', '').astype('float')
#check for NaNs
print(whisky_df.alcohol.isna().any())

False


In [22]:
whisky_df.rename(columns = {'alcohol': 'ABV'}, inplace = True)
print(whisky_df.columns)

Index(['name', 'nose', 'palate', 'finish', 'description', 'region',
       'distillery', 'bottler', 'age', 'ABV', 'maturation', 'chill_filter',
       'cask_strength'],
      dtype='object')


#### cask_strength and chill_filter have Yes/No entries when populated. Let's convert these to Boolean.

In [23]:
print({'cask_strength_unique': whisky_df.cask_strength.unique(), 'chill_filter_unique': whisky_df.chill_filter.unique()})

{'cask_strength_unique': array([nan, 'No', 'Yes'], dtype=object), 'chill_filter_unique': array([nan, 'No', 'Yes'], dtype=object)}


In [24]:
whisky_df.loc[:, ['cask_strength', 'chill_filter']] = whisky_df[['cask_strength', 'chill_filter']].replace({'Yes': True, 'No': False})

In [25]:
print({'cask_strength_unique': whisky_df.cask_strength.unique(), 'chill_filter_unique': whisky_df.chill_filter.unique()})

{'cask_strength_unique': array([nan, False, True], dtype=object), 'chill_filter_unique': array([nan, False, True], dtype=object)}


## Free text columns: Description, taste, nose, and palate

We want to extract information about the whiskies from these columns. Some of this info will be concrete (is it cask strength? is it chill filtered? which barrels was it aged in?). Some of that info is in the description but was not put into the bottle detail data on the website...so we'll need to extract it from free text and put it into the appropriate existing dataframe column. 

In [26]:
# we're going to import necessary NLP libraries

import spacy
from spacy.tokens import Doc, Span, Token # for creating global objects 
from spacy.matcher import Matcher # for rule-based matching
from spacy.language import Language # for building custom pipeline components

from copy import deepcopy


In [27]:
Doc.set_extension('name', force = True, default = None)

In [28]:
nlp = spacy.load('en_core_web_lg') # loads our NLP model

We want to batch process all the description text. So we'll construct a pipeline. We also want to keep track of what description belongs to what whisky. This will enable us to rejoin any results from NLP into our pandas dataframe. So we will construct a custom attribute via context creation.

In [29]:
# passing context metadata into attributes via pipeline requires the data to be in a specific form. We'll create a function to do transform the data into the right form.
# then the data will be used to create a Doc object generator based off of the text and context metadata.

def doc_context_tupling(data, textcolname, *args):
    col_list = [textcolname]
    attr_list = list(args)
    
    col_list.extend(attr_list)

    # data is in dataframe w/ form of whisky_df
    subset_dict_list = list(data[col_list].T.to_dict().values())

    # now let's form a (text, context_dictionary) tuple which is what the spacy pipeline requires. 

    data_context_list = [ (subset_dict.pop(textcolname), subset_dict) for subset_dict in subset_dict_list]

    # for each key-value pair in args, we'll need to create a Doc extension attribute.
    

    for doc, context in nlp.pipe(data_context_list, as_tuples = True):
        Doc.set_extension('context', force = True, default = context)
        yield doc


We are going to use some domain knowledge. It is unlikely that a whisky producer or reviewer would mention chill filtration in a description unless the whisky was NOT chill filtered. Chill filtration is a process that removes some esthers and fatty acids that can become non-soluble in lower ABV whisky at cooler temperatures. The process can prevent clouding but many connosieurs believe that it degrades the quality of the whisky and removes some of its mouthfeel and complexity. In any case, saying that your whisky is non chill-filtered is a point of pride. We'll thus create a very simple rule based matcher for this and not worry too heavily about the possibility that the review is saying that it IS chill filtered. The same logic goes for whether a whisky is cask strength or not.

In [31]:

matcher = Matcher(nlp.vocab)
pattern1 = [{'LOWER': 'chill'}, {'IS_PUNCT': True, 'OP': "?"},  {'LEMMA': 'filtration'} ]
pattern2 = [{'LOWER': 'chill'}, {'IS_PUNCT': True, 'OP': "?"},  {'LEMMA': 'filter'} ]
matcher.add('CHILL_FILTER', [pattern1, pattern2])

matcher2 = Matcher(nlp.vocab)
pattern3 = [{'LOWER': 'cask'}, {'IS_PUNCT': True, 'OP': "?"},  {'LEMMA': 'strength'} ]
matcher2.add('CASK_STRENGTH', [pattern3])



In [32]:
def update_cs_cf_info(data):

    descript_corpus = doc_context_tupling(data, 'description', 'name')

    new_data = deepcopy(data)

    for doc in descript_corpus:

        matches_chillf = matcher(doc)
        matches_cask = matcher2(doc)
        
        if len(matches_cask) > 0:
            new_data.loc[new_data.name == doc._.context['name'], 'cask_strength'] =  True 
        
        counter = 0
        if len(matches_chillf) > 0:
            new_data.loc[new_data.name == doc._.context['name'], 'chill_filter'] =  False 

    return new_data



In [33]:
new_whisk = update_cs_cf_info(whisky_df)

In [34]:
new_whisk.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8484 entries, 0 to 14141
Data columns (total 13 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   name           8484 non-null   object 
 1   nose           8484 non-null   object 
 2   palate         8484 non-null   object 
 3   finish         8484 non-null   object 
 4   description    8484 non-null   object 
 5   region         8484 non-null   object 
 6   distillery     8484 non-null   object 
 7   bottler        8484 non-null   object 
 8   age            6556 non-null   float64
 9   ABV            8484 non-null   float64
 10  maturation     1612 non-null   object 
 11  chill_filter   819 non-null    object 
 12  cask_strength  1643 non-null   object 
dtypes: float64(2), object(11)
memory usage: 1.2+ MB


The description also has information about the wood / type of barrel(s) that the whisky was aged in.  We want to be able to extract that information from the free text. There are different names used for 'casks' in the corpus. Some examples are octaves, pipes, butts, puncheons, etc. There are technical differences between these (size, shape, etc) that can have effects on the whisky taste. But it'll be good to create a broad named entity class for this.

Then there will be characteristics of the cask that directly impact flavor profiles such as whether the cask once held sherry, bourbon, etc or the kind of wood when relevant (oak can impart strong flavors to a Scotch).

In [35]:
dc = nlp(new_whisk.iloc[12].description)
print(dc)

Here we have the flagship single malt from Balvenie's series of Port Wood whiskies. This 21 year old expression, a marriage of rare Balvenie, was impressively finished in 30 year old port pipes. Originally released in 1996, this whisky is a veritable masterclass in poise and balance. It’s fruity and very subtly smoky, the perfect after-dinner dram.


In [36]:
@Language.component("cask_entity_creator")
def cask_entity_creator(doc):

    # matcher for entity creation
    cask_syn_matcher = Matcher(nlp.vocab)
    pattern_csk = [{'LEMMA': 'cask'} ]
    cask_syn_matcher.add('CSK', [pattern_csk])

    
    matches = cask_syn_matcher(doc)
    spans = [ Span(doc, start, end, label = "CSK") for match_id, start, end in matches]

    orig_ents = list(doc.ents)

    doc.set_ents(spans, default="unmodified")


    return doc



In [38]:
from sklearn.model_selection import train_test_split

In [40]:
from spacy.pipeline import EntityRuler
    
hp_chars = list(nlp.pipe(new_whisk.description))


In [195]:
from spacy.lang.en import English
nlp2 = spacy.load('en_core_web_lg')
ruler = nlp2.add_pipe("entity_ruler")

+6

#pattern_csk1 = [{"POS": "NOUN", "OP":"*"}, {"LEMMA": "cask"}]

pattern_csk1 = [{"POS": "NOUN", "OP":"*"}, {"LEMMA": {"IN": ["cask", "octave", "pipe", "puncheon", "butt", "barrel"]} } ]
pattern_csk2 = [{"POS": "NOUN", "OP":"*"}, {"POS": "CCONJ"}, {"POS": "NOUN", "OP":"*"}, {"LEMMA": {"IN": ["cask", "octave", "pipe", "puncheon", "butt"]} } ]


patterns = [ {"label": "CSK", "pattern": pattern_csk1 }, {"label": "CSK", "pattern": pattern_csk2 }, {"label": "WOD", "pattern": pattern_csk2 } ]


ruler.add_patterns(patterns)


In [196]:
doc = nlp2(new_whisk.iloc[159].description)

In [197]:
doc.ents

(the Tamnavulin Distillery,
 American,
 oak barrels,
 three,
 sherry casks,
 Three,
 Christmas,
 Speyside)

In [198]:
Tasting Notes

SyntaxError: invalid syntax (Temp/ipykernel_6968/1019552579.py, line 1)

In [201]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [202]:
vect = TfidfVectorizer()

In [215]:
Q = vect.fit_transform(new_whisk.nose.values)

In [216]:
first_document_vector=Q[0]

In [217]:
feature_names = vect.get_feature_names()
first_document_vector.todense()

matrix([[0., 0., 0., ..., 0., 0., 0.]])

In [218]:
print(feature_names)

['10', '11', '12', '15', '15yo', '16', '17', '18', '1940', '2006', '22', '28', '30', '35', '40', '57', '5th', '69', 'aberfeldy', 'aberlour', 'able', 'abound', 'abounds', 'about', 'above', 'abricot', 'abrupt', 'abruptly', 'absolutely', 'abundance', 'abv', 'acacia', 'accentuate', 'accompanied', 'accompanies', 'accompanying', 'acetone', 'acidic', 'acidity', 'across', 'act', 'acting', 'action', 'active', 'actors', 'actual', 'actually', 'add', 'added', 'adding', 'addition', 'additional', 'adds', 'admirable', 'aeration', 'aforementioned', 'after', 'afternoon', 'afterthought', 'afterwards', 'again', 'against', 'agave', 'age', 'aged', 'aggressive', 'agreement', 'agricole', 'ahead', 'ahh', 'air', 'airiness', 'airy', 'akin', 'alaska', 'alcohol', 'ale', 'align', 'alive', 'all', 'allowed', 'allowing', 'allows', 'allsorts', 'allspice', 'alluring', 'almond', 'almonds', 'almost', 'aloe', 'along', 'alongside', 'alpine', 'already', 'alright', 'also', 'although', 'always', 'amalfi', 'amaretti', 'amarett